In [1]:
import pandas as pd
from functools import reduce

In [91]:
bank_account_info_df = pd.read_csv("bank_account_info.csv")
loan_account_info_df = pd.read_csv("loan_account_info.csv")
motor_vehicle_info_from_borrower_df = pd.read_csv("motor_vehicle_info_from_borrower.csv")
fuel_emission_factor_based_on_engine_capacity_df = pd.read_csv("fuel_ef_look_up_based_on_engine_capacity.csv")
fuel_efficiency_based_on_make_and_model_df = pd.read_csv("fuel_efficiency_look_up_based_on_make_and_model.csv")
regional_statistical_distance_data_df = pd.read_csv("regional_statistical_distance_data.csv")
local_statistical_distance_data_df = pd.read_csv("local_statistical_distance_data.csv")

#Declaring default values
data_quality_score_one = "Score 1"
data_quality_score_two = "Score 2"
data_quality_score_three = "Score 3"
data_quality_score_four = "Score 4"
data_quality_score_five = "Score 5"

#Cleansing the input dataframes
loan_account_info_df['outstanding_amount'] = loan_account_info_df['outstanding_amount'].fillna(0)
loan_account_info_df['total_sanctioned_loan_amount_for_vehicle'] = loan_account_info_df['total_sanctioned_loan_amount_for_vehicle'].fillna(1)

motor_vehicle_info_from_borrower_df['units_consumed_per_month'] = motor_vehicle_info_from_borrower_df['units_consumed_per_month'].fillna(0)
motor_vehicle_info_from_borrower_df['estimated_fuel_consumption_per_month'] = motor_vehicle_info_from_borrower_df['estimated_fuel_consumption_per_month'].fillna(0)
motor_vehicle_info_from_borrower_df['estimated_distance_travel_per_month'] = motor_vehicle_info_from_borrower_df['estimated_distance_travel_per_month'].fillna(0)

#Merging DFs

input_dfs = [loan_account_info_df, motor_vehicle_info_from_borrower_df, fuel_efficiency_based_on_make_and_model_df]
combined_input_dfs = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='inner'), input_dfs)

combined_input_1_df = pd.DataFrame()
combined_input_1_df = combined_input_dfs[['account_holder_type','account_type','total_sanctioned_loan_amount_for_vehicle',
                    'loan_disbursed_year','outstanding_amount','tenure_in_years','monthly_payment_amount','past_due_payments_in_months',
                    'purpose_of_vehicle','units_consumed_per_month','estimated_fuel_consumption_per_month','estimated_distance_travel_per_month',
                    'country','vehicle_type',]].copy()

combined_input_1_df['make'] = combined_input_dfs['make_x'].copy()
combined_input_1_df['model'] = combined_input_dfs['model_x'].copy()
combined_input_1_df['fuel_type'] = combined_input_dfs['fuel_type_x'].copy()
combined_input_1_df['version'] = combined_input_dfs['version_x'].copy() 
combined_input_1_df['engine_capacity'] = (combined_input_dfs['engine_capacity_x'].astype(int)).copy()
combined_input_1_df['efficiency_of_vehicle'] = combined_input_dfs['efficiency_of_vehicle_x'].copy() 
combined_input_1_df['car_year'] = combined_input_dfs['car_year_x'].copy()

combined_input_1_df['vehicle_type'] = combined_input_dfs['vehicle_type_x'].copy()
combined_input_1_df['vehicle_mass'] = combined_input_dfs['vehicle_mass_x'].copy()
combined_input_1_df['loan_account_number'] = combined_input_dfs['loan_account_number_x'].copy()
combined_input_1_df['country'] = combined_input_dfs['country_x'].copy()
combined_input_1_df['reporting_financial_company_name'] = combined_input_dfs['reporting_financial_company_name_x'].copy()
combined_input_1_df['account_holder_name'] = combined_input_dfs['account_holder_name_x'].copy()
combined_input_1_df['account_number'] = combined_input_dfs['account_number_x'].copy()

combined_input_1_df['engine_capacity'] = combined_input_1_df['engine_capacity'].apply(lambda x: 1350 if (x > 0 and x <=1350) else x)
combined_input_1_df['engine_capacity'] = combined_input_1_df['engine_capacity'].apply(lambda x: 1600 if (x > 1350 and x <=1600) else x)
combined_input_1_df['engine_capacity'] = combined_input_1_df['engine_capacity'].apply(lambda x: 2000 if (x > 1600 and x <=2000) else x)
combined_input_1_df['engine_capacity'] = combined_input_1_df['engine_capacity'].apply(lambda x: 3000 if (x > 2000 and x <=3000) else x)
combined_input_1_df['engine_capacity'] = combined_input_1_df['engine_capacity'].apply(lambda x: 15000 if (x > 3000 and x <=15000) else x)

modified_fuel_emifact_based_on_engine_capacity_df = pd.DataFrame()
modified_fuel_emifact_based_on_engine_capacity_df['fuel_type'] = fuel_emission_factor_based_on_engine_capacity_df['fuel_type'].copy()
modified_fuel_emifact_based_on_engine_capacity_df['engine_capacity'] = fuel_emission_factor_based_on_engine_capacity_df['max_cc_range'].copy()
modified_fuel_emifact_based_on_engine_capacity_df['emission_factor'] = fuel_emission_factor_based_on_engine_capacity_df['emission_factor'].copy()
#modified_fuel_emifact_based_on_engine_capacity_df['unit'] = fuel_emission_factor_based_on_engine_capacity_df['unit'].copy()

combined_input_2_df = pd.merge(combined_input_1_df, modified_fuel_emifact_based_on_engine_capacity_df, on=['fuel_type','engine_capacity'] , how = 'left')

#Calculating Attribution factor
temp = pd.DataFrame()
temp1 = pd.DataFrame()
temp = combined_input_2_df
temp.loc[combined_input_2_df['total_sanctioned_loan_amount_for_vehicle'] !=1, 'attribution_factor'] = combined_input_2_df['outstanding_amount'].div(combined_input_2_df['total_sanctioned_loan_amount_for_vehicle'])
temp.loc[combined_input_2_df['total_sanctioned_loan_amount_for_vehicle'] == 1, 'attribution_factor'] = 0

temp.loc[temp['estimated_fuel_consumption_per_month'] != 0, 'Emission_CO2e'] = (temp['attribution_factor'].mul(temp['estimated_fuel_consumption_per_month'])).mul(temp['emission_factor'])
temp.loc[temp['Emission_CO2e'] == 0, 'Emission_CO2e'] = ((temp['attribution_factor'].mul(temp['estimated_distance_travel_per_month'])).mul(temp['emission_factor'])).mul(temp['efficiency_of_vehicle'])
temp['Emission_CO2e'] = temp['Emission_CO2e'].fillna(0)
temp.loc[temp['Emission_CO2e'] != 0, 'data_quality'] = data_quality_score_one

temp1 = pd.merge(temp, regional_statistical_distance_data_df, on=['country'], how = 'left')
temp1.loc[temp1['Emission_CO2e'] == 0, 'Emission_CO2e_3'] = ((temp1['attribution_factor'].mul(temp1['avg_distance_travelled_per_person_per_month'])).mul(temp1['emission_factor'])).mul(temp1['efficiency_of_vehicle'])
temp1['Emission_CO2e_3'] = temp1['Emission_CO2e_3'].fillna(0)
temp1.loc[temp1['Emission_CO2e_3'] != 0, 'data_quality_3'] = data_quality_score_three
temp1.loc[temp1['Emission_CO2e'] == 0, 'Emission_CO2e'] = temp1['Emission_CO2e_3']
temp1.loc[temp1['data_quality'].isnull(), 'data_quality'] = temp1['data_quality_3']
temp1.drop(['Emission_CO2e_3'], axis=1)
temp1.drop(['data_quality_3'], axis=1)

temp = temp1

temp2 = pd.merge(temp1, local_statistical_distance_data_df, on=['country'], how = 'left')
temp2.loc[temp2['Emission_CO2e'] == 0, 'Emission_CO2e_2'] = ((temp2['attribution_factor'].mul(temp2['avg_loc_distance_travelled_per_person_per_month'])).mul(temp2['emission_factor'])).mul(temp2['efficiency_of_vehicle'])
temp2['Emission_CO2e_2'] = temp2['Emission_CO2e_2'].fillna(0)
temp2.loc[temp2['Emission_CO2e_2'] != 0, 'data_quality_2'] = data_quality_score_two
temp2.loc[temp2['Emission_CO2e'] == 0, 'Emission_CO2e'] = temp2['Emission_CO2e_2']
temp2.loc[temp2['data_quality'].isnull(), 'data_quality'] = temp2['data_quality_2']
temp2.drop(['Emission_CO2e_2'], axis=1)
temp2.drop(['data_quality_2'], axis=1)

temp = temp2

final_out = pd.DataFrame()
final_out = temp[['account_holder_type', 'account_type',
       'total_sanctioned_loan_amount_for_vehicle', 'loan_disbursed_year',
       'outstanding_amount', 'tenure_in_years', 'monthly_payment_amount',
       'past_due_payments_in_months', 'purpose_of_vehicle',
       'units_consumed_per_month', 'estimated_fuel_consumption_per_month',
       'estimated_distance_travel_per_month', 'country', 'vehicle_type',
       'make', 'model', 'fuel_type', 'version', 'engine_capacity',
       'efficiency_of_vehicle', 'car_year', 'vehicle_mass',
       'loan_account_number', 'reporting_financial_company_name',
       'account_holder_name', 'account_number', 'emission_factor',
       'attribution_factor', 'Emission_CO2e', 'data_quality', 
       'avg_distance_travelled_per_person_per_month', 
       'avg_loc_distance_travelled_per_person_per_month']].copy()

final_out['continent'] = temp['continent_x'].copy()
final_out['unit'] = temp['unit_x'].copy()
final_out['source'] = temp['source_x'].copy()

final_out.to_csv("out.csv")
combined_input_2_df.to_csv("combined_input_dfs.csv")
print("Done")


<bound method IndexOpsMixin.tolist of Index(['account_holder_type', 'account_type',
       'total_sanctioned_loan_amount_for_vehicle', 'loan_disbursed_year',
       'outstanding_amount', 'tenure_in_years', 'monthly_payment_amount',
       'past_due_payments_in_months', 'purpose_of_vehicle',
       'units_consumed_per_month', 'estimated_fuel_consumption_per_month',
       'estimated_distance_travel_per_month', 'country', 'vehicle_type',
       'make', 'model', 'fuel_type', 'version', 'engine_capacity',
       'efficiency_of_vehicle', 'car_year', 'vehicle_mass',
       'loan_account_number', 'reporting_financial_company_name',
       'account_holder_name', 'account_number', 'emission_factor',
       'attribution_factor', 'Emission_CO2e', 'data_quality', 'continent_x',
       'avg_distance_travelled_per_person_per_month', 'unit_x', 'source_x',
       'Emission_CO2e_3', 'data_quality_3', 'continent_y',
       'avg_loc_distance_travelled_per_person_per_month', 'unit_y', 'source_y',
       '